# Build Modelling Dataset

## Set Up

In [1]:
%matplotlib inline

import logging
import cPickle as pickle
import pandas as pd
import numpy as np
import time

from datetime import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Clean Stations

In [36]:
readings = pickle.load(open('data/parsed/readings_dataset_utc.p', 'rb'))
stations = pickle.load(open('data/parsed/stations_dataset_final.p', 'rb')).set_index('Id')

readings.sort_values(by=['Id', 'Timestamp'], inplace=True)
readings.set_index(['Id', 'Timestamp'], inplace=True)

In [37]:
# remove stations that have less than 21 days of data
stations_to_remove = []
for station_id in readings.index.get_level_values('Id').unique():
    nb_days = len(np.unique(readings.loc[station_id].index.strftime('%Y-%m-%d')))
    if nb_days < 21:
        stations_to_remove.append(station_id)
    
readings.drop(stations_to_remove, inplace=True)
stations.drop(stations_to_remove, inplace=True)
print 'Removed stations %s' % stations_to_remove

Removed stations [u'BikePoints_237', u'BikePoints_497', u'BikePoints_796', u'BikePoints_805', u'BikePoints_807', u'BikePoints_809', u'BikePoints_811', u'BikePoints_817']


## Closest Stations

In [38]:
from geopy.distance import vincenty

# get the locations of all the stations in tuples
stations_locations = pd.Series(zip(stations.Latitude, stations.Longitude), index=stations.index)

# create a DF to hold the nearest stations
surrounding_stations = pd.DataFrame(columns=['Near%d' % (i+1) for i in range(0,20)], index=stations.index)
for station_id, station_loc in stations_locations.iteritems():
    ranking = stations_locations.apply(lambda x: vincenty(station_loc, x).kilometers).sort_values()
    surrounding_stations.loc[station_id] = ranking.index.values[1:21]

## Resample Weather

In [39]:
weather = pickle.load(open('data/parsed/weather_dataset_utc.p', 'rb'))

weather.set_index(['Timestamp'], inplace=True)
weather = weather.resample('5min').mean().ffill()

## Preprocess Redistribution

In [40]:
distributed = pickle.load(open('data/parsed/distributed_dataset_final.p', 'rb'))
distributed = distributed.query('NbBikes != 0').drop(['Name'], axis=1)
distributed.columns = ['DistNbBikes']

In [41]:
collected = pickle.load(open('data/parsed/collected_dataset_final.p', 'rb'))
collected = collected.query('NbBikes != 0').drop(['Name'], axis=1)
collected.columns = ['CollNbBikes']

## Merge Readings

In [42]:
dfs = []
for station_id in readings.index.get_level_values('Id').unique():
    # resample with a freq of 5 mins
    resampled_readings = readings.loc[station_id].resample('5min').mean().ffill()
    
    # merge weather
    merged = resampled_readings.merge(weather, how='left', left_index=True, right_index=True)
    
    # merge distributed
    if station_id in distributed.index:
        dist_resampled = distributed.loc[station_id].resample('5min').sum()
        merged = merged.merge(dist_resampled, how='left', left_index=True, right_index=True)
        
    # merge collected
    if station_id in collected.index:
        coll_resampled = collected.loc[station_id].resample('5min').sum()
        merged = merged.merge(coll_resampled, how='left', left_index=True, right_index=True)
        
    # merge surrounding
    surrounding_stations_nbbikes = []
    for surrounding_station_id in surrounding_stations.loc[station_id].values:
        surrounding_stations_nbbikes.append(readings.loc[surrounding_station_id].NbBikes.resample('5min').mean().ffill())
    surrounding_stations_nbbikes = pd.concat(surrounding_stations_nbbikes, axis=1)
    surrounding_stations_nbbikes.columns = surrounding_stations.columns
    merged = merged.merge(surrounding_stations_nbbikes, how='left', left_index=True, right_index=True)
    
    # set the id
    merged['Id'] = station_id

    dfs.append(merged.reset_index())

readings_weather = pd.concat(dfs, ignore_index=True)

### Reduce Column Size

In [43]:
start_time = time.time()

readings_weather.NbBikes = readings_weather.NbBikes.astype('int16')
readings_weather.NbDocks = readings_weather.NbDocks.astype('int16')
readings_weather.NbEmptyDocks = readings_weather.NbEmptyDocks.astype('int16')
readings_weather.NbUnusableDocks = readings_weather.NbUnusableDocks.astype('int16')
readings_weather.CollNbBikes = readings_weather.CollNbBikes.astype('float16')
readings_weather.DistNbBikes = readings_weather.DistNbBikes.astype('float16')

readings_weather.DistNbBikes = readings_weather.DistNbBikes.fillna(0).astype('int16')
readings_weather.CollNbBikes = readings_weather.CollNbBikes.fillna(0).astype('int16')

# have to use float due to pandas gotcha with NaN in integer 
for col in surrounding_stations.columns:
    readings_weather[col] = readings_weather[col].astype('float16')
    
readings_weather.DewPt = readings_weather.DewPt.astype('float16')
readings_weather.Humidity = readings_weather.Humidity.astype('float16')
readings_weather.Pressure = readings_weather.Pressure.astype('float16')
readings_weather.Temp = readings_weather.Temp.astype('float16')
readings_weather.Visibility = readings_weather.Visibility.astype('float16')
readings_weather.WindDirD = readings_weather.WindDirD.astype('float16')
readings_weather.WindSpeed = readings_weather.WindSpeed.astype('float16')

readings_weather.Fog = readings_weather.Fog.astype('int8')
readings_weather.Rain = readings_weather.Rain.astype('int8')
readings_weather.Hail = readings_weather.Hail.astype('int8')
readings_weather.Snow = readings_weather.Snow.astype('int8')
readings_weather.Thunder = readings_weather.Thunder.astype('int8')
readings_weather.Tornado = readings_weather.Tornado.astype('int8')

end_time = time.time()
print 'Modifying data types took %s' % (end_time - start_time)

Modifying data types took 5.24216604233


### Modify Timestamp

In [44]:
start_time = time.time()

readings_weather['Holiday'] = readings_weather.Timestamp.apply(lambda x: x.month == 5 and x.day == 30).astype('int8')
readings_weather['Weekday'] = readings_weather.Timestamp.apply(lambda x: x.dayofweek < 5).astype('int8')
readings_weather['Weekend'] = readings_weather.Timestamp.apply(lambda x: x.dayofweek > 4).astype('int8')
readings_weather['TimeOfYear'] = readings_weather.Timestamp.apply(lambda x: (x - datetime(2016,1,1)).total_seconds())
readings_weather['TimeOfDay'] = readings_weather.Timestamp.apply(lambda x: (x - x.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds())
readings_weather['WeekOfYear'] = readings_weather.Timestamp.apply(lambda x: x.isocalendar()[1])
readings_weather['DayOfWeek'] = readings_weather.Timestamp.apply(lambda x: x.weekday())

end_time = time.time()
print 'Changing timestamp took %s' % (end_time - start_time)

Changing timestamp took 398.124156952


### Re-arrange

In [45]:
readings_weather.sort_values(by=['Id', 'Timestamp'], inplace=True)
readings_weather.set_index(['Id', 'Timestamp'], inplace=True)

In [46]:
pickle.dump(readings_weather, open("data/parsed/readings_resampled.p", "wb"))
print 'Saved'

Saved


### Autoregressive

In [2]:
readings_weather = pickle.load(open('data/parsed/readings_resampled.p', 'rb'))

In [4]:
def shift_cols(df, cols, periods, mode):
    for period in periods:    
        target_cols = []
        for col in cols:
            label = '%s%s%d' % (col, mode, period)
            target_cols.append(label)
            df[label] = df[col]
    
        station_ids = df.index.get_level_values('Id').unique()   
        for station_id in station_ids:   
            shifted = df.loc[station_id, target_cols].shift(periods=period)
            df.loc[station_id, target_cols] = shifted.values

In [5]:
start_time = time.time()

shift_cols(readings_weather, ['Temp', 'Humidity', 'Rain', 'Fog'], [2, 12], 'TMinus')

readings_weather.TempTMinus2 = readings_weather.TempTMinus2.astype('float16')
readings_weather.TempTMinus12 = readings_weather.TempTMinus12.astype('float16')
readings_weather.HumidityTMinus2 = readings_weather.HumidityTMinus2.astype('float16')
readings_weather.HumidityTMinus12 = readings_weather.HumidityTMinus12.astype('float16')
readings_weather.RainTMinus2 = readings_weather.RainTMinus2.astype('float16')
readings_weather.RainTMinus12 = readings_weather.RainTMinus12.astype('float16')
readings_weather.FogTMinus2 = readings_weather.FogTMinus2.astype('float16')
readings_weather.FogTMinus12 = readings_weather.FogTMinus12.astype('float16')

end_time = time.time()
print 'Adding previous weather columns took %s' % (end_time - start_time)

Adding previous weather columns took 234.748711109


In [6]:
start_time = time.time()

shift_cols(readings_weather, ['NbBikes'], [1, 2, 3, 12, 18], 'TMinus')

readings_weather.NbBikesTMinus1 = readings_weather.NbBikesTMinus1.astype('float16')
readings_weather.NbBikesTMinus2 = readings_weather.NbBikesTMinus2.astype('float16')
readings_weather.NbBikesTMinus3 = readings_weather.NbBikesTMinus3.astype('float16')
readings_weather.NbBikesTMinus12 = readings_weather.NbBikesTMinus12.astype('float16')
readings_weather.NbBikesTMinus18 = readings_weather.NbBikesTMinus18.astype('float16')

end_time = time.time()
print 'Adding autoregressive columns took %s' % (end_time - start_time)

Adding autoregressive columns took 321.069705009


In [7]:
start_time = time.time()
shift_cols(readings_weather, ['Near1'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near2'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near3'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near4'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near5'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near6'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near7'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near8'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near9'], [2, 3, 12, 18], 'TMinus')
shift_cols(readings_weather, ['Near10'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near11'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near12'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near13'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near14'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near15'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near16'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near17'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near18'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near19'], [2, 3, 12, 18], 'TMinus')
#shift_cols(readings_weather, ['Near20'], [2, 3, 12, 18], 'TMinus')
end_time = time.time()

print 'Adding autoregressive columns took %s' % (end_time - start_time)

Adding autoregressive columns took -0.0148899555206


In [8]:
pickle.dump(readings_weather, open("data/parsed/readings_autoregressive.p", "wb"))
print 'Saved'

Saved


## Add Historic Avg

In [2]:
readings_weather = pickle.load(open('data/parsed/readings_autoregressive.p', 'rb'))

In [3]:
start_time = time.time()

readings_weather['HistAvg'] = np.NaN

for station_id in readings_weather.index.get_level_values('Id').unique():
    window = readings_weather.loc[station_id]
    
    station_hist_avgs = window.groupby(['Holiday', 'Weekday', 'TimeOfDay'])[['NbBikes']].mean()    
    
    station_ts = window.reset_index().set_index(['Holiday', 'Weekday', 'TimeOfDay'])[['Timestamp']]
    hist_avg_ts = station_ts.merge(station_hist_avgs, how='inner', left_index=True, right_index=True)
    hist_avg_ts = hist_avg_ts.reset_index(drop=True).set_index('Timestamp').sort_index().round().NbBikes

    readings_weather.loc[station_id].HistAvg = hist_avg_ts
    
readings_weather.HistAvg = readings_weather.HistAvg.astype('float16')

end_time = time.time()
print 'Adding historic average column took %s' % (end_time - start_time)

/home/jfconavarrete/Documents/Work/Config/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Adding historic average column took 92.1469268799


In [4]:
import math

readings_weather.Diff = readings_weather.NbBikes - readings_weather.HistAvg
readings_weather.Diff = readings_weather.Diff.apply(math.fabs)
readings_weather.Diff.describe()

count    9.352339e+06
mean     4.326587e+00
std      3.752515e+00
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      6.000000e+00
max      4.500000e+01
dtype: float64

## Add Cumulative Redistribution

In [7]:
def sum_cumulative(series, n):
    cumulative_sum = series
    for i in range(1, n + 1):
        cumulative_sum = series.shift(i).fillna(0) + cumulative_sum
    return cumulative_sum.values

In [8]:
start_time = time.time()

readings_weather['CollNbBikesCum2'] = np.NaN
readings_weather['CollNbBikesCum6'] = np.NaN
readings_weather['DistNbBikesCum2'] = np.NaN
readings_weather['DistNbBikesCum6'] = np.NaN

for station_id in readings_weather.index.get_level_values('Id').unique():
    window = readings_weather.loc[station_id]

    collected = window.CollNbBikes
    readings_weather.loc[station_id].CollNbBikesCum2 = sum_cumulative(collected, 2)
    readings_weather.loc[station_id].CollNbBikesCum6 = sum_cumulative(collected, 6)

    distributed = window.DistNbBikes
    readings_weather.loc[station_id].DistNbBikesCum2 = sum_cumulative(distributed, 2)
    readings_weather.loc[station_id].DistNbBikesCum6 = sum_cumulative(distributed, 6)
    
readings_weather.CollNbBikesCum2 = readings_weather.CollNbBikesCum2.astype('int16')
readings_weather.CollNbBikesCum6 = readings_weather.CollNbBikesCum6.astype('int16')
readings_weather.DistNbBikesCum2 = readings_weather.DistNbBikesCum2.astype('int16')
readings_weather.DistNbBikesCum6 = readings_weather.DistNbBikesCum6.astype('int16')
    
end_time = time.time()
print 'Adding cumulative redistribution columns took %s' % (end_time - start_time)

Adding cumulative redistribution columns took 207.053358793


In [9]:
pickle.dump(readings_weather, open("data/parsed/readings_full.p", "wb"))
print 'Saved'

Saved


## Clean

Delete stations which experienced periods of inactivity.

In [2]:
readings_weather = pickle.load(open('data/parsed/readings_full.p', 'rb'))

In [3]:
missing_days = pickle.load(open('data/parsed/missing_days.p', 'rb'))

In [9]:
invalid_readings = []
for idx, row in missing_days.iterrows():
    if idx not in readings_weather.index:
        continue
    
    for missing_day in row.MissingDays:
        labels = readings_weather.loc[idx].loc[missing_day:missing_day.replace(hour=23, minute=59, second=59, microsecond=999999)].index
        for label in labels:
            invalid_readings.append((idx, label))

readings_weather.drop(invalid_readings, inplace=True)
print 'Removed %d invalid readings' % len(invalid_readings)

Removed 27648 invalid readings


## Save

In [10]:
readings_weather.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9324691 entries, (BikePoints_1, 2016-05-15 13:35:00+00:00) to (BikePoints_99, 2016-06-26 23:25:00+00:00)
Data columns (total 104 columns):
CollNbBikes         int16
DewPt               float16
DistNbBikes         int16
Fog                 int8
Hail                int8
Humidity            float16
NbBikes             int16
NbDocks             int16
NbEmptyDocks        int16
NbUnusableDocks     int16
Near1               float16
Near10              float16
Near11              float16
Near12              float16
Near13              float16
Near14              float16
Near15              float16
Near16              float16
Near17              float16
Near18              float16
Near19              float16
Near2               float16
Near20              float16
Near3               float16
Near4               float16
Near5               float16
Near6               float16
Near7               float16
Near8               float16
Near9            

In [11]:
pickle.dump(readings_weather, open("data/parsed/readings_clean.p", "wb"))
print 'Saved'

Saved
